# This file is seeing what we get when we just apply the Random Forest directly to the Time Series

In [7]:
# Imports
import sys
import pandas as pd
import numpy as np
from rfgap import RFGAP


# Get the data
sys.path.insert(0, '/yunity/arusty/PF-GAP')

static2024 = pd.read_csv('../data/static2024.csv')
time_series = pd.read_csv('../data/time_series.csv')
labels = pd.read_csv('../data/labels.csv')


In [8]:
# Combine the static data with the time series data
static2024.shape, time_series.shape

((503, 11), (503, 829))

In [10]:
train = np.concat([static2024, time_series], axis=1)
train.shape

(503, 840)

In [15]:
rfgap = RFGAP(prediction_type="classification", y = labels, oob_score = True)
rfgap.fit(train, labels)

/yunity/arusty/PF-GAP/.venv/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [17]:
rfgap.oob_score_

0.7912524850894632